# Hyperparameter optimization

## Outline
   
- **Hyperparameter tuning and CV using Bayesian Optimizer** 
    - Scikit-optimize provides a drop-in replacement for GridSearchCV, which utilizes Bayesian Optimization where a predictive model referred to as "surrogate" is used to model the search space and utilized to arrive at good parameter values combination as soon as possible.<br>

- **Find Parameters for the following tree based models**
    - XGBOOST
    - LightGBM
    - Random Forest
    
- **Save tuned parameters**

In [4]:
import pandas as pd
import numpy  as np
 
# # Models
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold

from scripts.preprocess import get_feature_list_from_text_excluding_target
from scripts.utils import set_seed
from scripts.utils import save_dict_as_json
from scripts.utils import read_json_file

seed = 112358

set_seed(seed)

## Data

In [6]:
full_data = pd.read_csv(r"data/prepared_data_for_modeling.csv")
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23485 entries, 0 to 23484
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   datetime                    23485 non-null  object 
 1   campaign_id                 23485 non-null  int64  
 2   spend                       23485 non-null  float64
 3   impressions                 23485 non-null  float64
 4   clicks                      23485 non-null  float64
 5   purchase                    23485 non-null  float64
 6   hour                        23485 non-null  int64  
 7   unique_ids                  23485 non-null  object 
 8   first_24_hour               23485 non-null  bool   
 9   test_set                    23485 non-null  bool   
 10  ctr                         23485 non-null  float64
 11  conversion_rate             23485 non-null  float64
 12  custom_conversion_rate      23485 non-null  float64
 13  good_performance            234

Specify target variable and get features

In [7]:
target = 'next_hour_good_performance'
sample_features = get_feature_list_from_text_excluding_target(r"data/feature_lists_for_modeling.txt", target)
sample_features

Successfully returned list from:  data/feature_lists_for_modeling.txt


['spend',
 'purchase',
 'impressions',
 'hour',
 'good_performance',
 'dayofweek',
 'custom_conversion_rate',
 'conversion_rate',
 'clicks',
 'campaign_id']

Take full data for optimization

In [8]:
X = np.ascontiguousarray(full_data[full_data['first_24_hour']][sample_features])
y = np.ascontiguousarray(full_data[full_data['first_24_hour']][target])

In [9]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

### XGBoost

#### Hpo

In [21]:
ITERATIONS = 100

# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = XGBClassifier(
        n_jobs = 1,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx'
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (5, 500),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'n_estimators': (50, 500),
        'scale_pos_weight': (1, 50, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(n_splits=3,
                         shuffle=True,
                         random_state=seed),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = seed
)

Start finding parameters and save result

In [22]:
xgb_tuning_result = bayes_cv_tuner.fit(X, y, callback=status_print)
save_dict_as_json('HPO/XGB/', light_gbm_tuned.best_params_, 'best_params')

Load best params found

In [2]:
best_params = read_json_file('HPO/XGB/best_params.json')
best_params

{'colsample_bylevel': 0.8015579071911014,
 'colsample_bytree': 0.44364889457651413,
 'gamma': 3.811128976537413e-05,
 'learning_rate': 0.2700390206185342,
 'max_delta_step': 18,
 'max_depth': 36,
 'min_child_weight': 4,
 'n_estimators': 83,
 'reg_alpha': 1.5057560255472018e-06,
 'reg_lambda': 659.4262286994368,
 'scale_pos_weight': 26,
 'subsample': 0.8835665823899177}

## b. LightGBM

#### HPO

In [18]:
ITERATIONS = 100

bayes_cv_tuner = BayesSearchCV(
    estimator = lgb.LGBMClassifier(
        n_jobs = 1,
        objective = 'binary',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx'
    ),
    
    search_spaces = {
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'learning_rate': (0.001, 1.0, 'log-uniform'),
        'max_depth': (1, 500),
        'max_bin':(50,5000),
        'n_estimators': (50, 100),
        'reg_lambda': (1e-9, 1000.0, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1, 100, 'log-uniform'),
        'subsample': (0.01, 1.0, 'uniform'),
        "boosting_type": ["gbdt", "dart"],
        "num_leaves": (100, 2000),
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=seed
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = seed
)

Start finding parameters and save result

In [187]:
light_gbm_tuned = bayes_cv_tuner.fit(X, y, callback=status_print)
save_dict_as_json('HPO/LightGBM/', light_gbm_tuned.best_params_, 'best_params')

Load best params found

In [37]:
best_params = read_json_file('HPO/LightGBM/best_params.json')
best_params

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.4194735998017874,
 'learning_rate': 0.3713670265402593,
 'max_bin': 4761,
 'max_depth': 19,
 'n_estimators': 60,
 'num_leaves': 2000,
 'reg_alpha': 1.0,
 'reg_lambda': 1000.0,
 'scale_pos_weight': 32,
 'subsample': 0.7029212166466253}

## c. Random Forest

In [66]:
ITERATIONS = 100

# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = RandomForestClassifier(
        n_jobs = -1
    ),
    search_spaces = {
    'min_samples_split': (2, 10), 
    'n_estimators': (10, 1000),
    'max_depth': (3, 100),
    'max_features': ['auto', 'sqrt'],
    'class_weight': ['balanced', 'balanced_subsample'],
    'criterion': ['gini', 'entropy']
},    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=seed
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = seed
)

In [67]:
rf_tuned = bayes_cv_tuner.fit(X, y, callback=status_print)
save_dict_as_json('HPO/RandomForest/', rf_tuned.best_params_, 'best_params')

F:\miniconda3\envs\lfi\lib\site-packages\sklearn\utils\validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Model #1
Best ROC-AUC: 0.7285
Best params: OrderedDict([('class_weight', 'balanced'), ('criterion', 'entropy'), ('max_depth', 93), ('max_features', 'auto'), ('min_samples_split', 7), ('n_estimators', 420)])



F:\miniconda3\envs\lfi\lib\site-packages\sklearn\utils\validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Model #2
Best ROC-AUC: 0.7391
Best params: OrderedDict([('class_weight', 'balanced_subsample'), ('criterion', 'entropy'), ('max_depth', 32), ('max_features', 'sqrt'), ('min_samples_split', 9), ('n_estimators', 72)])



F:\miniconda3\envs\lfi\lib\site-packages\sklearn\utils\validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


...

F:\miniconda3\envs\lfi\lib\site-packages\skopt\optimizer\optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
F:\miniconda3\envs\lfi\lib\site-packages\sklearn\utils\validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Model #99
Best ROC-AUC: 0.8212
Best params: OrderedDict([('class_weight', 'balanced_subsample'), ('criterion', 'entropy'), ('max_depth', 4), ('max_features', 'sqrt'), ('min_samples_split', 9), ('n_estimators', 1000)])



F:\miniconda3\envs\lfi\lib\site-packages\sklearn\utils\validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Model #100
Best ROC-AUC: 0.8212
Best params: OrderedDict([('class_weight', 'balanced_subsample'), ('criterion', 'entropy'), ('max_depth', 4), ('max_features', 'sqrt'), ('min_samples_split', 9), ('n_estimators', 1000)])

Successfully saved results in HPO\RandomForest


F:\miniconda3\envs\lfi\lib\site-packages\sklearn\utils\validation.py:931: FutureWarning: Passing attributes to check_is_fitted is deprecated and will be removed in 0.23. The attributes argument is ignored.
  warnings.warn("Passing attributes to check_is_fitted is deprecated"


Load best params found

In [4]:
best_params = read_json_file('HPO/RandomForest/best_params.json')
best_params

{'class_weight': 'balanced_subsample',
 'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'sqrt',
 'min_samples_split': 9,
 'n_estimators': 1000}